In [159]:
from keras.models import Sequential , Model
from keras.layers import Dense , Conv2D , MaxPool2D , BatchNormalization , Reshape , Input , Flatten
from keras.layers.advanced_activations import LeakyReLU

from keras.optimizers import Adam

from keras.datasets import mnist

In [160]:
import os

import matplotlib as plt
import numpy as np

%matplotlib inline

In [161]:
WIDTH = 28 #96
HEIGHT = 28
CHANNEL = 1 #3

LATENT_DIM = 100 #latent variable z

BATCH_SIZE = 32
EPOCHS = 10

PATH = 'faces/'

#生成多少个图像 长*宽
ROW = 5
COL = 5

In [162]:
load_index = 0

images_name = os.listdir(PATH)

IMAGES_COUNT = len(images_name)

In [163]:
(X_train , y_train),(X_test , y_test) = mnist.load_data()
X_train = X_train/127.5-1
X_train = np.expand_dims(X_train , 3)

In [164]:
def load_mnist():
    return X_train[np.random.randint(0, X_train.shape[0], BATCH_SIZE)]
    
def write_image_mnist(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    generated_image = generator_i.predict(noise)
    generated_image = generated_image*0.5+0.5
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    
    count=0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count,:,:,0] , cmap = 'gray')
            axes[i][j].axis('off')
            count += 1
            
    fig.savefig('images/No.%d.png' % epoch)
    plt.pyplot.close()

In [165]:
def load_image(batch_size = BATCH_SIZE):
    global load_index
    
    images = []
    
    for i in range(batch_size):
        images.append(plt.image.imread(PATH + images_name[(load_index + i) % IMAGES_COUNT]))
    
    load_index += batch_size
    
    return np.array(images)/255

def write_image(epoch):
    
    noise = np.random.normal(size = (ROW*COL , LATENT_DIM))
    generated_image = generator_i.predict(noise)
    generated_image *= 255
    
    fig , axes = plt.pyplot.subplots(ROW , COL)
    
    count=0
    
    for i in range(ROW):
        for j in range(COL):
            axes[i][j].imshow(generated_image[count])
            count += 1
            
    fig.savefig('images/No.%d.png' % epoch)
    plt.pyplot.close()
    
    #plt.image.imsave('images/'+str(epoch)+'.jpg')

In [166]:
def generator():
    #sample from noise z
    model = Sequential(name='generator')
    
    model.add(Dense(units = 256 , input_dim = LATENT_DIM , name='dense1'))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8 , name='batchnorm1'))
    model.add(Dense(units = 512 , name='dense2'))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8 , name='batchnorm2'))
    model.add(Dense(units = 1024 , name='dense3'))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8 , name='batchnorm3'))
    #model.add(Dense(units = 5000 , activation='relu' , name='dense4'))
    #model.add(BatchNormalization(name='batchnorm4'))
    #model.add(Dense(units = 10000 , activation='relu' , name='dense5'))
    #model.add(BatchNormalization(name='batchnorm5'))
    #model.add(Dense(units = 20000 , activation='relu' , name='dense6'))
    #model.add(BatchNormalization(name='batchnorm6'))
    model.add(Dense(units = WIDTH*HEIGHT*CHANNEL , activation='tanh' , name='dense7'))
    model.add(Reshape(target_shape=(WIDTH , HEIGHT , CHANNEL) , name='reshape1'))
    
    #model.summary()
    
    noise = Input(shape=(LATENT_DIM , ) , name='input1')
    image = model(noise)
    
    return Model(noise , image , name='generator_Model')

In [167]:
def discriminator():
    #input a image to discriminate real or fake
    model = Sequential(name='discriminator')
    
    model.add(Flatten(input_shape = (WIDTH,HEIGHT,CHANNEL) , name='flatten1'))
    #model.add(Dense(units=10000 , name='dense8'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu1'))
    #model.add(Dense(units=5000 , name='dense9'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu2'))
    #model.add(Dense(units=1024 , name='dense10'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu3'))
    model.add(Dense(units=512 , name='dense11'))
    model.add(LeakyReLU(0.2 , name='leakyrelu4'))
    model.add(Dense(units=256 , name='dense12'))
    model.add(LeakyReLU(0.2 , name='leakyrelu5'))
    #model.add(Dense(units=32 , name='dense13'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu6'))
    #model.add(Dense(units=4 , name='dense14'))
    #model.add(LeakyReLU(0.3 , name='leakyrelu7'))
    model.add(Dense(units=1 , activation='sigmoid' , name='dense15'))
    
    #model.summary()
    
    image = Input(shape=(WIDTH , HEIGHT , CHANNEL) , name='input1')
    validity = model(image)
    
    return Model(image , validity , name='discriminator_Model')

In [156]:
def combined_model(generator_i , discriminator_i):
    #生成器和判别器组合成整体
    z = Input(shape=(LATENT_DIM , ) , name='z')
    
    image = generator_i(z)
    discriminator_i.trainable = False
    validity = discriminator_i(image)
    
    return Model(z , validity , name='combined_model')

In [168]:
discriminator_i = discriminator()
discriminator_i.compile(optimizer=Adam(lr=0.0002 , beta_1=0.5) , loss='binary_crossentropy' , metrics=['accuracy'])

generator_i = generator()

combined_model_i = combined_model(generator_i , discriminator_i)


combined_model_i.compile(optimizer=Adam(lr=0.0002 , beta_1=0.5) , loss='binary_crossentropy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten1 (Flatten)           (None, 784)               0         
_________________________________________________________________
dense11 (Dense)              (None, 512)               401920    
_________________________________________________________________
leakyrelu4 (LeakyReLU)       (None, 512)               0         
_________________________________________________________________
dense12 (Dense)              (None, 256)               131328    
_________________________________________________________________
leakyrelu5 (LeakyReLU)       (None, 256)               0         
_________________________________________________________________
dense15 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

In [169]:
real_labels = np.ones(shape=(BATCH_SIZE , 1)) #真实样本label为1
fake_labels = np.zeros(shape=(BATCH_SIZE , 1)) #假样本label为0

for i in range(20000):
    noise = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))

    #real_image = load_image()
    real_image = load_mnist()
    #训练判别器
    fake_image = generator_i.predict(noise)

    real_loss = discriminator_i.train_on_batch(real_image , real_labels)
    fake_loss = discriminator_i.train_on_batch(fake_image , fake_labels)

    loss = np.add(real_loss , fake_loss)/2

    #训练生成器
    noise2 = np.random.normal(size=(BATCH_SIZE , LATENT_DIM))
    generator_loss = combined_model_i.train_on_batch(noise2 , real_labels)
    
    #print(loss , generator_loss)
    
    if i % 100 == 0:
        #write_image(i)
        write_image_mnist(i)

C:\Anaconda3\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[0.9467044 0.0625   ] 0.77934575
[0.39060977 0.828125  ] 0.8805331
[0.32878986 0.84375   ] 0.9756985
[0.2740297 0.90625  ] 1.0545326
[0.23990297 0.921875  ] 1.2486203
[0.18346857 1.        ] 1.322283
[0.16381007 1.        ] 1.4223003
[0.13993905 1.        ] 1.5253472
[0.13289781 1.        ] 1.6350505
[0.10964542 1.        ] 1.7806125
[0.11815085 1.        ] 1.8227206
[0.09424974 1.        ] 1.9786818
[0.10535213 1.        ] 1.9160267
[0.09635839 1.        ] 2.0490716
[0.08680885 1.        ] 2.1664948
[0.08030829 1.        ] 2.3093061
[0.07591477 1.        ] 2.3505611
[0.05258251 1.        ] 2.3930392
[0.05619407 1.        ] 2.4657054
[0.05702867 1.        ] 2.40842
[0.05102511 1.        ] 2.5738635
[0.0517558 1.       ] 2.6760604
[0.05192693 1.        ] 2.681529
[0.04893949 1.        ] 2.6898048
[0.04045117 1.        ] 2.6824865
[0.03772424 1.        ] 2.734176
[0.04097608 1.        ] 2.9076896
[0.03400218 1.        ] 2.9049413
[0.04015389 1.        ] 2.9063995
[0.03312024 1.        ] 

[0.11767134 0.984375  ] 4.3936887
[0.06571187 1.        ] 4.651698
[0.1505757 0.984375 ] 3.8989873
[0.0709296 0.984375 ] 4.2972803
[0.05123332 1.        ] 4.3673
[0.21373987 0.9375    ] 3.736846
[0.07441086 0.984375  ] 4.5933475
[0.41616884 0.8125    ] 3.4890313
[0.08119603 0.96875   ] 4.781601
[1.1150029 0.5625   ] 2.4534388
[0.12521632 0.921875  ] 3.7326875
[0.05401889 1.        ] 3.9251933
[0.08317237 1.        ] 4.291748
[0.09627058 0.984375  ] 4.523151
[0.29130995 0.859375  ] 3.56466
[0.07015877 0.984375  ] 4.380508
[0.13922991 0.953125  ] 4.710211
[0.18002206 0.921875  ] 3.7514782
[0.17914107 0.921875  ] 4.6194353
[0.7145826 0.6875   ] 3.585074
[0.03742567 1.        ] 4.4545784
[0.45725167 0.796875  ] 3.2630792
[0.102957 1.      ] 3.814889
[0.5427045 0.765625 ] 2.8591402
[0.07081226 0.984375  ] 3.9625843
[0.31004316 0.890625  ] 2.7983725
[0.27297652 0.84375   ] 4.3626237
[0.53628266 0.734375  ] 3.3843663
[0.07347526 1.        ] 3.9898853
[0.3184923 0.875    ] 3.056107
[0.13448206

[0.72155535 0.390625  ] 0.7677555
[0.77191097 0.40625   ] 0.75602067
[0.6698525 0.453125 ] 0.7802414
[0.78889656 0.296875  ] 0.7043042
[0.69179344 0.484375  ] 0.7162063
[0.72960293 0.40625   ] 0.7346216
[0.6904301 0.46875  ] 0.7607713
[0.7090548 0.453125 ] 0.75112426
[0.6643283 0.484375 ] 0.77959967
[0.69655854 0.5       ] 0.7677289
[0.7487879 0.359375 ] 0.7434392
[0.65912676 0.546875  ] 0.75773036
[0.72875786 0.296875  ] 0.7589943
[0.7032523 0.484375 ] 0.75470185
[0.713673 0.40625 ] 0.70760953
[0.65906554 0.515625  ] 0.7458018
[0.64980507 0.5625    ] 0.7986885
[0.73099124 0.359375  ] 0.76470244
[0.67445815 0.5       ] 0.76246154
[0.673065 0.53125 ] 0.7553992
[0.6778351 0.484375 ] 0.7635731
[0.7226447 0.390625 ] 0.7282105
[0.7053473 0.5      ] 0.71267104
[0.69685984 0.46875   ] 0.7066896
[0.7182446 0.390625 ] 0.72574925
[0.7202643 0.40625  ] 0.7368728
[0.6735637 0.484375 ] 0.6960355
[0.69662064 0.421875  ] 0.693912
[0.6980451 0.4375   ] 0.7111144
[0.65268123 0.5       ] 0.7372333
[0.69

[0.5542171 0.875    ] 0.78428745
[0.64116204 0.609375  ] 0.81649446
[0.6260859 0.703125 ] 0.80033016
[0.64020336 0.65625   ] 0.7931962
[0.63233894 0.5625    ] 0.802166
[0.6173156 0.734375 ] 0.80595267
[0.6566932 0.609375 ] 0.7962835
[0.61997545 0.59375   ] 0.8054811
[0.61956227 0.703125  ] 0.8072169
[0.6591543 0.609375 ] 0.7890866
[0.655154 0.625   ] 0.7849597
[0.6377357 0.625    ] 0.8012055
[0.6399544 0.5625   ] 0.8025024
[0.6354884 0.640625 ] 0.78720284
[0.6309656 0.625    ] 0.788638
[0.63423216 0.609375  ] 0.79519314
[0.63312775 0.609375  ] 0.7386402
[0.6718395 0.546875 ] 0.73414755
[0.6104818 0.75     ] 0.76226616
[0.64713967 0.6875    ] 0.72378075
[0.64484745 0.640625  ] 0.74047446
[0.62409127 0.703125  ] 0.7714689
[0.64893585 0.640625  ] 0.76286334
[0.6452302 0.59375  ] 0.7833222
[0.6310019 0.703125 ] 0.7514785
[0.6822218 0.578125 ] 0.72683716
[0.62358254 0.65625   ] 0.7350985
[0.63892746 0.671875  ] 0.7666282
[0.64096916 0.59375   ] 0.7383056
[0.62688255 0.640625  ] 0.77350724
[

...

[0.7032291 0.53125  ] 1.0474368
[0.62117815 0.671875  ] 1.0736587
[0.6042081 0.671875 ] 1.1328115
[0.6553369 0.609375 ] 1.126538
[0.68193305 0.59375   ] 1.0074573
[0.61566615 0.65625   ] 1.0028275
[0.6313312 0.609375 ] 1.0251461
[0.68003464 0.59375   ] 0.9989967
[0.6213243 0.671875 ] 1.102001
[0.6151225 0.625    ] 1.0973212
[0.6965867 0.578125 ] 0.96847814
[0.6536516 0.546875 ] 0.8648755
[0.6282877 0.65625  ] 1.0004432
[0.5977193 0.703125 ] 1.0774941
[0.69160944 0.578125  ] 1.1592999
[0.7091893 0.5625   ] 1.1145194
[0.6344757 0.609375 ] 1.1387358
[0.61934966 0.703125  ] 1.0431542
[0.6618449 0.625    ] 1.1237688
[0.6854007 0.65625  ] 0.98149884
[0.6429042 0.640625 ] 0.9111411
[0.6325637 0.578125 ] 1.1101925
[0.6844529 0.640625 ] 1.0938824
[0.61287737 0.59375   ] 1.0832491
[0.6466496 0.609375 ] 1.1257545
[0.65158796 0.640625  ] 1.0709827
[0.735072 0.515625] 0.9522297
[0.5294961 0.765625 ] 1.1290518
[0.5884323 0.71875  ] 1.0794235
[0.6726355 0.53125  ] 0.9718037
[0.6279902 0.640625 ] 0.95

[0.609902 0.59375 ] 1.0345292
[0.6565714 0.609375 ] 0.9501292
[0.6361408 0.609375 ] 1.0238242
[0.6749505 0.640625 ] 1.0053337
[0.70504105 0.625     ] 0.97248447
[0.60188043 0.734375  ] 1.1093411
[0.523998 0.859375] 1.0723786
[0.63725376 0.640625  ] 1.1685177
[0.67174643 0.609375  ] 1.0155289
[0.63033605 0.625     ] 0.89091796
[0.6556474 0.65625  ] 0.86771667
[0.66300225 0.65625   ] 0.9343406
[0.58540165 0.671875  ] 1.043648
[0.6337853 0.578125 ] 0.9999026
[0.5975497 0.6875   ] 1.0165766
[0.63448614 0.65625   ] 0.8930976
[0.5595521 0.765625 ] 0.9312104
[0.7690253 0.578125 ] 1.0010717
[0.6079006 0.640625 ] 1.0095011
[0.61663806 0.625     ] 1.0293839
[0.6983259 0.609375 ] 1.1213316
[0.64198107 0.609375  ] 0.9380189
[0.57491994 0.671875  ] 0.9937426
[0.63977444 0.65625   ] 0.9446348
[0.6985785 0.546875 ] 0.96411806
[0.61140025 0.640625  ] 1.0945244
[0.615978 0.6875  ] 0.9268534
[0.63725466 0.65625   ] 0.95869756
[0.63451505 0.59375   ] 0.9790614
[0.6352952 0.703125 ] 1.0088769
[0.7228203 0

[0.62453127 0.59375   ] 0.91440463
[0.636351 0.640625] 0.9777665
[0.60212207 0.703125  ] 0.96518856
[0.67723024 0.59375   ] 0.9865798
[0.59189314 0.640625  ] 1.1315427
[0.62780315 0.671875  ] 0.97364306
[0.55608886 0.765625  ] 1.0337384
[0.59382474 0.640625  ] 1.0333432
[0.70700496 0.546875  ] 1.0238969
[0.5328094 0.78125  ] 0.9465719
[0.7210636 0.578125 ] 0.86205274
[0.69062936 0.609375  ] 0.84757257
[0.6722978 0.59375  ] 0.955791
[0.6544635 0.625    ] 0.9698783
[0.56531894 0.703125  ] 1.1001343
[0.6148713 0.625    ] 1.0969461
[0.59487987 0.71875   ] 0.98997545
[0.59105295 0.6875    ] 0.902107
[0.63792974 0.59375   ] 0.9245885
[0.67811954 0.640625  ] 0.9725435
[0.6955743 0.546875 ] 0.9913178
[0.5742713 0.71875  ] 1.0878377
[0.59720016 0.65625   ] 1.0747032
[0.6078931 0.640625 ] 1.1985457
[0.67735255 0.578125  ] 0.9520346
[0.668876 0.578125] 1.0039341
[0.65579474 0.5625    ] 1.0118389
[0.6081645 0.59375  ] 0.9585308
[0.5741892 0.71875  ] 1.0797927
[0.63861907 0.671875  ] 1.0566118
[0.5

[0.59056115 0.703125  ] 1.0660985
[0.5823325 0.65625  ] 0.9141238
[0.6040591 0.65625  ] 1.0408428
[0.6324533 0.609375 ] 0.96402854
[0.6415268 0.640625 ] 1.0264933
[0.6383238 0.640625 ] 1.0231174
[0.68781483 0.546875  ] 1.1591576
[0.6715919 0.640625 ] 1.184188
[0.7073902 0.5      ] 0.8857976
[0.5630986 0.71875  ] 0.9712295
[0.5938047 0.703125 ] 0.885544
[0.5760343 0.734375 ] 0.93738616
[0.60546446 0.6875    ] 1.004727
[0.704162 0.546875] 0.9894239
[0.6222334 0.65625  ] 0.99709064
[0.6555196 0.578125 ] 1.0544912
[0.6316719 0.59375  ] 0.92880917
[0.61826926 0.671875  ] 1.1771741
[0.58486617 0.71875   ] 0.9986854
[0.6684394 0.625    ] 1.0596874
[0.642575 0.625   ] 1.0130398
[0.63221836 0.671875  ] 0.99886465
[0.6073974 0.703125 ] 0.94317234
[0.63876545 0.703125  ] 1.1285248
[0.72516876 0.546875  ] 1.1006181
[0.5811144 0.671875 ] 1.1680284
[0.6010959 0.703125 ] 1.1195977
[0.5792978 0.703125 ] 1.1066993
[0.6407108 0.578125 ] 1.0311093
[0.648139 0.671875] 0.9711814
[0.6801425 0.578125 ] 0.914

[0.59687006 0.6875    ] 0.9043437
[0.6578536 0.578125 ] 0.98492116
[0.71545666 0.625     ] 1.0951924
[0.6694871 0.578125 ] 0.9085673
[0.5979711 0.75     ] 1.0978823
[0.62482274 0.65625   ] 1.1160693
[0.55688226 0.734375  ] 1.049984
[0.6785377 0.65625  ] 0.9741222
[0.6892989 0.53125  ] 0.9135134
[0.6105113 0.6875   ] 1.0300287
[0.66453445 0.5625    ] 1.0637336
[0.7304003 0.53125  ] 0.96801174
[0.6249147 0.625    ] 1.0449312
[0.6197138 0.71875  ] 1.0690298
[0.6229859 0.671875 ] 1.045203
[0.6522354 0.65625  ] 0.90927625
[0.6312317 0.625    ] 1.0550237
[0.65692437 0.625     ] 1.0602615
[0.66337454 0.625     ] 1.0025924
[0.6007998 0.65625  ] 1.072763
[0.62461704 0.609375  ] 1.0096593
[0.63235986 0.625     ] 0.9535076
[0.6160221 0.6875   ] 0.99621254
[0.6019422 0.734375 ] 1.0657833
[0.63260156 0.640625  ] 1.0729125
[0.6867951 0.59375  ] 0.94904745
[0.5601405 0.703125 ] 1.1220053
[0.5510623 0.703125 ] 1.1636238
[0.61155915 0.6875    ] 1.2326045
[0.6823944 0.59375  ] 1.1171987
[0.66133046 0.60

In [53]:
discriminator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 28, 28, 1)         0         
_________________________________________________________________
discriminator (Sequential)   (None, 1)                 533505    
Total params: 533,505
Trainable params: 0
Non-trainable params: 533,505
_________________________________________________________________


In [54]:
generator_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          (None, 100)               0         
_________________________________________________________________
generator (Sequential)       (None, 28, 28, 1)         1097744   
Total params: 1,097,744
Trainable params: 1,095,184
Non-trainable params: 2,560
_________________________________________________________________


In [55]:
combined_model_i.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z (InputLayer)               (None, 100)               0         
_________________________________________________________________
generator_Model (Model)      (None, 28, 28, 1)         1097744   
_________________________________________________________________
discriminator_Model (Model)  (None, 1)                 533505    
Total params: 1,631,249
Trainable params: 1,095,184
Non-trainable params: 536,065
_________________________________________________________________
